# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


In [ ]:
import pandas as pd

In [ ]:
import requests
response = requests.get("https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31",headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'})
response

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
soup.find_all('p')

In [ ]:
print(soup.prettify())

In [ ]:
movie_number_list = soup.find_all("h3", class_= "ipc-title__text")
movie_number_list

In [ ]:
# Initialize lists to store movie names and positions
movie_names = []
positions = []

# Find all movie components
components = soup.find_all("h3", class_="ipc-title__text")

for component in components:
    # Extract movie name
    movie_name = component.text.strip()
    movie_names.append(movie_name)
    
    # Extract position
    position = movie_name.split(".")[0]  # Extract the position from the movie name
    positions.append(position)

# Create DataFrame
df = pd.DataFrame({'movie_name': movie_names, 'position': positions})
df['movie_name'] = df['movie_name'].str.split(r'^\d+\.\s', expand=True)[1]
df

In [ ]:
title_list = soup.find_all("h3", class_= "ipc-title__text")
title_list

In [ ]:
# Initialize lists to store movie names and positions
titles = []

# Find all movie components
components = soup.find_all("h3", class_="ipc-title__text")

for component in components:
    # Extract movie name
    title = component.text.strip()
    titles.append(title)
# Create DataFrame
df = pd.DataFrame({'title': titles})
df['title'] = df['title'].str.split(r'^\d+\.\s', expand=True)[1]
df

In [ ]:
year = soup.find_all("span", class_= "sc-b0691f29-8 ilsLEX dli-title-metadata-item")
year

In [ ]:
import re
# Initialize lists to store movie names and positions
years = []

# Find all movie components
components = soup.find_all("span", class_= "sc-b0691f29-8 ilsLEX dli-title-metadata-item")

for component in components:
    # Extract movie name
    year = component.text.strip()
    years.append(year)

# Create DataFrame
df = pd.DataFrame({'year': years})
df = df[df['year'].between(1990, 1992)]
df['year'] = df['year'].astype(int)
df

# Convert 'year' column to integer type


# Filter DataFrame for years between 1990 and 1992
#df = df[df['year'].filter(1990, 1991, 1992)]


In [ ]:
import re

# Extract movie years and convert to integer type, then filter for years between 1990 and 1992
df = pd.DataFrame({
    'year': [int(re.search(r'\d+', component.text.strip()).group()) for component in soup.find_all("span", class_="sc-b0691f29-8 ilsLEX dli-title-metadata-item") if re.search(r'\d+', component.text.strip())]
}).query('1990 <= year <= 1992')
df

In [ ]:
rating = soup.find_all("span", class_= "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
rating

In [ ]:
# Initialize lists to store movie names and positions
ratings = []

# Find all movie components
components = soup.find_all("span", class_= "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")

for component in components:
    # Extract movie name
    rating = component.text.strip()
    rating_float=float(rating.split()[0])
    ratings.append(rating_float)
# Create DataFrame
df = pd.DataFrame({'rating': ratings})
df

In [ ]:
# Initialize lists to store movie ratings and votes
ratings = []
votes = []

# Find all movie components
components = soup.find_all("span", class_= "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")

for component in components:
    # Extract movie rating and votes
    rating = component.text.strip().split()[0]  # Extract the rating
    try:
        rating_float = float(rating)  # Convert the rating to float
    except ValueError:
        rating_float = None  # Set to None if conversion fails
    ratings.append(rating_float)
    
    vote = component.text.strip().split('(')[-1].split(')')[0]  # Extract the votes
    votes.append(vote)

# Create DataFrame
df = pd.DataFrame({'rating': ratings, 'votes': votes})

# Display the DataFrame
df


In [ ]:
duration = soup.find_all("span", class_= "sc-b0691f29-8 ilsLEX dli-title-metadata-item")
duration

In [ ]:
import re

# Initialize a list to store movie durations
durations = []

# Find all movie components
components = soup.find_all("span", class_= "sc-b0691f29-8 ilsLEX dli-title-metadata-item")

# Define a regular expression pattern to match the time format (e.g., "1h 58m")
time_pattern = r"\d+h\s*\d+m"

# Extract movie durations
for component in components:
    # Extract text from the component
    text = component.get_text(strip=True)
    # Check if the text matches the time pattern
    if re.match(time_pattern, text):
        durations.append(text)

# Create DataFrame
df = pd.DataFrame({'duration': durations})

# Display the DataFrame
df


In [ ]:
soup.find_all('a', class_ = 'ipc-title-link-wrapper')

In [ ]:
import requests
href_list = []
for i in soup.find_all('a', class_ = 'ipc-title-link-wrapper'):
    href_list.append(i.get("href"))

all_actors = []

for i in href_list:
    response = requests.get('https://www.imdb.com'+i, headers = headers)
    soup = BeatifulSoup(response.content)
    actors = [actor.get.text() for actor in soup.find_all('a', attrs = {'data-testid': 'title-cast-item__actor'})]
    all_actors.append(actors)

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get('https://www.imdb.com/', headers=headers)


In [ ]:
import requests
from bs4 import BeautifulSoup
headers = requesr.get('https://www.imdb.com/title/tt0102926/?ref_=sr_t_1', headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get('https://www.imdb.com/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

href_list = []
for i in soup.find_all('a', class_='ipc-title-link-wrapper'):
    href_list.append(i.get("href"))

all_actors = []

for i in href_list:
    response = requests.get('https://www.imdb.com'+i, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    actors = [actor.get_text() for actor in soup.find_all('a', attrs={'data-testid': 'title-cast-item__actor'})]
    all_actors.append(actors)


In [ ]:
import requests
from bs4 import BeautifulSoup

# Define the headers
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Make a request to the IMDb homepage
response = requests.get('https://www.imdb.com/', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract href links
href_list = [i.get("href") for i in soup.find_all('a', class_='ipc-title-link-wrapper')]

# Initialize list to store actors
all_actors = []

# Iterate over each href link
for href in href_list:
    # Make a request to the movie page
    response = requests.get('https://www.imdb.com' + href, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract actors from the cast section
    actors = [actor.get_text() for actor in soup.find_all('a', attrs={'data-testid': 'title-cast-item__actor'})]
    
    # Append the list of actors to the all_actors list
    all_actors.append(actors)


In [ ]:
title-cast-item_actor

In [ ]:
all_actors

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_imdb_info(url, class_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    components = soup.find_all(class_=class_name)
    info_list = [component.text.strip() for component in components]
    return info_list

def scrape_imdb(title_type, user_rating, start_date, end_date):
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&user_rating={user_rating}-&release_date={start_date},{end_date}"
    
    movie_names = scrape_imdb_info(url, "ipc-title__text")
    titles = scrape_imdb_info(url, "ipc-title__text")
    years = scrape_imdb_info(url, "sc-b0691f29-8 ilsLEX dli-title-metadata-item")
    ratings = scrape_imdb_info(url, "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
    durations = scrape_imdb_info(url, "sc-b0691f29-8 ilsLEX dli-title-metadata-item")
    
    df = pd.DataFrame({
        'movie_name': movie_names,
        'title': titles,
        'year': years,
        'rating': ratings,
        'duration': durations
    })
    
    return df

# Example usage:
title_type = "feature"
user_rating = 7.5
start_date = "1990-01-01"
end_date = "1992-12-31"

df = scrape_imdb(title_type, user_rating, start_date, end_date)
print(df)
df

**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [ ]:
# Your solution goes here

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here